Create an excel file, enter the name of each moon in column A. Enter the semimajor axis in column B; make sure they’re all in the same unit and note down the unit. Enter the orbital period in column C; again make sure they’re all in the same unit and note down the unit. If the period from the source you used is given as a negative number, disregard the minus sign as it just means the revolution is retrograde. In order to use the equation discussed, all quantities need to be in SI units. If the quantities you enter in columns B and C are already in SI units, no unit conversion is needed. However, mostly likely they are not. Therefore, let’s proceed under the assumption that unit conversion is needed. In the following operations, we’ll assume the semimajor axes in column B are all in km, and the orbital periods in column C are all in days. If the units from your source are different, make adjustments to the conversion factors. In column D (currently unoccupied), in cell D1 (corresponding to the first moon) type = B1 ∗ 1000 and press enter. This performs the conversion from km (column B) to m (column D) for the first moon. Now click on cell D1 so that the cell is highlighted with a thick rectangle. Hover the cursor over the cell until you see a little cross sign, then move the cursor towards the lower right corner of the cell, hold down the left mouse button and drag the rectangle down column D all the way to the last row that contains your data. All the cells in column D now should be filled with values corresponding to cells in column B with the conversion from km to m. Now let’s repeat the procedure to do unit conversion for the orbital period in column C (in days in this example). Go to column E (unoccupied for now), in cell E1 type = C1 ∗ 24 ∗ 60 ∗ 60 and press enter. This performs the conversion from days (column C) to seconds (column E) for the first moon. Now just like what we did before with column D, click on cell E1, hover over, and drag the rectangle all the way down to the last useful cell in column E to convert units for all other moons. Now we have semimajor axes R in meters stored in column D, and orbital period T in seconds stored in column E. Next we want to calculate the values of R3 in column F, and T2 in column G.

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('data.csv')
# Headers: Name,Orbital Period (days),Radius (km),Distance from Jupiter (km)

# Set Ganymede, Callisto, Io, and Europa as Galilean moons 
df['Galilean'] = df['Name'].isin(['Ganymede', 'Callisto', 'Io', 'Europa'])

#Convert Orbital Period to Seconds
df['Orbital Period (s)'] = df['Orbital Period (days)'] * 24 * 60 * 60

#Convert Radius to Meters
df['Radius (m)'] = df['Radius (km)'] * 1000

#Convert Distance from Jupiter to Meters
df['Distance from Jupiter (m)'] = df['Distance from Jupiter (km)'] * 1000

#Calculate R (Radius of orbit)
df['R (m)'] = df['Distance from Jupiter (m)'] + df['Radius (m)']

# Calculate R^3 and T^2
df['R3'] = df['R (m)'] ** 3
df['T2'] = df['Orbital Period (s)'] ** 2

In [3]:
df['R3'] = df['R (m)'] ** 3
df['T2'] = df['Orbital Period (s)'] ** 2

In [4]:
#Format the head to a markdown table
head = df.head() 
markdownHead = "| " + " | ".join(head.columns) + " |\n" + "| " + " | ".join(["---" for i in range(len(head.columns))]) + " |\n"
for i in range(len(head)):
    markdownHead += "| " + " | ".join([str(head.iloc[i][j]) for j in range(len(head.columns))]) + " |\n"

markdownHead

'| Name | Orbital Period (days) | Radius (km) | Distance from Jupiter (km) | Galilean | Orbital Period (s) | Radius (m) | Distance from Jupiter (m) | R (m) | R3 | T2 |\n| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |\n| Metis | 0.3 | 21.5 | 128000 | False | 25919.999999999996 | 21500.0 | 128000000 | 128021500.0 | 2.0982089455139382e+24 | 671846399.9999998 |\n| Adrastea | 0.3 | 8.2 | 129000 | False | 25919.999999999996 | 8200.0 | 129000000 | 129008200.0 | 2.1470983946224314e+24 | 671846399.9999998 |\n| Amalthea | 0.5 | 83.5 | 181400 | False | 43200.0 | 83500.0 | 181400000 | 181483500.0 | 5.977387881860633e+24 | 1866240000.0 |\n| Thebe | 0.68 | 49.3 | 221900 | False | 58752.0 | 49300.0 | 221900000 | 221949300.0 | 1.0933553615416017e+25 | 3451797504.0 |\n| Io | 1.77 | 1821.6 | 421800 | True | 152928.0 | 1821600.0 | 421800000 | 423621600.0 | 7.6021124363922526e+25 | 23386973184.0 |\n'

In [5]:
# Export to csv
df.to_csv('out.csv', index=False)

In [12]:
# Plot 
fig = px.scatter(df, x='R3', y='T2', hover_name='Name', color='Galilean')
fig.update_layout(
    title='Kepler\'s Third Law',
    xaxis_title='R^3 (m^3)',
    yaxis_title='T^2 (s^2)',
    showlegend=True
)

# Calculate the slope of the best fit line
m, b = np.polyfit(df['R3'], df['T2'], 1)

# Plot the best fit line and the equation
fig.add_trace(go.Scatter(x=df['R3'], y=m*df['R3']+b, mode='lines', name='y = {:.5e}x + {:.5e}'.format(m, b)))
fig.show()

# Calculate Mass of Jupiter
G = 6.67430e-11
print(m)
M = 4 * np.pi**2 / (G * m)
print('Mass of Jupiter: {:.5e} kg'.format(M))

3.146086440304132e-16
Mass of Jupiter: 1.88011e+27 kg


In [13]:
#Now do the same for just the Galilean moons
fig = px.scatter(df[df['Galilean']], x='R3', y='T2', hover_name='Name')
fig.update_layout(
    title='Kepler\'s Third Law for Galilean Moons',
    xaxis_title='R^3 (m^3)',
    yaxis_title='T^2 (s^2)',
    showlegend=True
)

# Calculate the slope of the best fit line
m, b = np.polyfit(df[df['Galilean']]['R3'], df[df['Galilean']]['T2'], 1)

# Plot the best fit line and the equation
fig.add_trace(go.Scatter(x=df[df['Galilean']]['R3'], y=m*df[df['Galilean']]['R3']+b, mode='lines', name='y = {:.5e}x + {:.5e}'.format(m, b)))
fig.show()

print(m)
# Calculate Mass of Jupiter
M = 4 * np.pi**2 / (G * m)
print('Mass of Jupiter (Galilean Moons): {:.5e} kg'.format(M))

3.104676779596539e-16
Mass of Jupiter (Galilean Moons): 1.90519e+27 kg
